In [1]:
import random
import re
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

In [2]:
model = BayesianModel()
comb_df = pd.DataFrame()

In [10]:
#creates the data with 19 modes wach for every image
#18 nodes are for 9 features of 2 images each
#last node represents whether the data is similar or dissimilar
#data is saved to the csv file : comb_df
#path is the path to the csv file for training
#path = 'AND_dataset/Dataset[WithFeatures]/AND_Features.csv'
def create_Data(path):
    df = pd.read_csv(path)
    ids = df['ImageId']
    id_list = [re.sub(r'\D', "", i)for i in ids]
    unique_ids = pd.unique(id_list)
    yes = pd.DataFrame(data = [1])
    no = pd.DataFrame(data = [0])
    yes = yes.reset_index(drop=True)
    no = no.reset_index(drop=True)
    k=-1
    all_ids = ids.tolist()
    comb_df = pd.DataFrame(np.zeros(19)).T
    comb_df.columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']
    for i in unique_ids:
        k=k+1;
        first_part = df.loc[df['ImageId'] == i+'a'].iloc[:,2:]
        second_part = df.loc[df['ImageId'] == i+'b'].iloc[:,2:]
        third_part = df.loc[df['ImageId'] == i+'c'].iloc[:,2:]
        
        if(first_part.empty == False):
            xx = first_part
        elif(second_part.empty == False):
            xx = second_part
        elif(third_part.empty == False):
            xx = third_part
            
        if(first_part.empty):
            first_part = xx
        if(second_part.empty):
            second_part = xx
        if(third_part.empty):
            third_part = xx
        
        first_part = first_part.reset_index(drop=True)
        second_part = second_part.reset_index(drop=True)
        third_part = third_part.reset_index(drop=True)
        
        row1 = pd.concat([pd.concat([first_part, second_part], axis=1, join='inner'), yes], axis = 1, join='inner')
        row2 = pd.concat([pd.concat([first_part, third_part], axis=1, join='inner'), yes], axis = 1, join='inner')
        final = pd.concat([row1, row2])
        final.columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']
        comb_df = comb_df.append(final)
        # next id
        mod_list = np.delete(unique_ids, k)
        for j in range(5):
            idx = random.choice(mod_list)
            rand_id = idx+'a'
            jj = 0;
            alphas = ['b','c']
            while rand_id not in all_ids:
                rand_id = idx+alphas[jj]
                jj = jj+1;
            jj=0
            rand_part = df.loc[df['ImageId'] == rand_id].iloc[:,2:]
            rand_part = rand_part.reset_index(drop=True)
            final = pd.concat([pd.concat([first_part, rand_part], axis=1, join='inner'), no], axis = 1, join='inner')
            final.columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']
            comb_df = comb_df.append(final)
            mod_list = np.delete(mod_list,idx)
            #print(final)
        print('processed'+i)
        comb_df.to_csv("9feat-siamese-dataset.csv")
    return comb_df

#creates the Bayesian Network of 19 nodes:
#this is used in training phase and used to generate the CPDs at all the nodes
def createBayesianModel(train_data):
    data = pd.DataFrame(data=train_data)
    model = BayesianModel([('f6', 'f2'), ('f6', 'f3'),('f6','f4'),('f3','f4'),('f3','f8'),('f4','f1'),('f4','f7'),('f1','f2'),('f7','f1'),('f7','f9'),('f8','f7'),('f8','f5'),('f5','f9'),('f2','f5'),('f15', 'f11'), ('f15', 'f12'),('f15','f13'),('f12','f13'),('f12','f17'),('f13','f10'),('f13','f16'),('f10','f11'),('f16','f10'),('f16','f18'),('f17','f16'),('f17','f14'),('f14','f18'),('f11','f14'),('f7','f19'),('f16','f19')])
    model.fit(data)
    model.get_cpds()
    for cpd in model.get_cpds():
        print(cpd)
    return model
    
#function to test the model
def test_model(data,model):
    test_data = data.reset_index(drop=True)
    test_data_1 = test_data.loc[test_data['verify_ab'] == 1].reset_index(drop=True)
    test_data_0 = test_data.loc[test_data['verify_ab'] == 0].reset_index(drop=True)

    print("Accuracy for combined DataSet is: ")
    predictedData, llr = getPredictionResult(test_data,model)
    predictedData = pd.DataFrame(predictedData)
    actualData = test_data['f19'].reshape(predictedData.shape)
    calcAccuracy(predictedData,actualData)

    print("Accuracy for Type1 Within Writer DataSet is:")
    predictedData, llr = getPredictionResult(test_data_1,model)
    predictedData = pd.DataFrame(predictedData)
    actualData = test_data_1['f19'].reshape(predictedData.shape)
    calcAccuracy(predictedData,actualData)
    
    print("Accuracy for Type2 Between Writer DataSet is: ")
    predictedData, llr = getPredictionResult(test_data_0,model)
    predictedData = pd.DataFrame(predictedData)
    actualData = test_data_0['f19'].reshape(predictedData.shape)
    calcAccuracy(predictedData,actualData)


def train_model(train_data):
    model = createBayesianModel(train_data)
    return model

# call this method with the path to training data csv to train the model
def PGM(path):
    #uncomment this to create data
    #comb_df = create_Data(path)
    
    #load the previously created data
    comb_df = pd.read_csv(path)
    #partitioned the data into training data and testing data
    train_data = comb_df.iloc[0:3000,:].astype(int) # if loading from csv [0:3000,1:]
    test_data = comb_df.iloc[3000:,:].astype(int)
    
    model = train_model(train_data)
    #this should be called on unknown data
#     test_model(test_data,model)
    return model

# path: path till PGMHiddenTest folder
def test_two_images(path, model):
    import pandas as pd
    
    # read csv for data pair
    imgList = pd.read_csv(path+"/PGMTestPairs.csv", sep=",")
    imgList = imgList.iloc[:,1:]
    imgList = imgList.reset_index(drop=True)
    
    # read pgm test data
    testData = pd.read_csv(path+'/PGMTestData.csv', sep=",")
    testData = testData.iloc[:,1:]
    testData = testData.reset_index(drop=True)
    
    comb_df = pd.DataFrame(columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18'])
    for id in range(imgList.shape[0]):
        im1 = testData.loc[testData['ImageId'] == str(imgList.iloc[id,0].split('.')[0])]
        im1 = im1.iloc[:,1:]
        im1 = im1.reset_index(drop=True)
        im2 = testData.loc[testData['ImageId'] == str(imgList.iloc[id,1].split('.')[0])]
        im2 = im2.iloc[:,1:]
        im2 = im2.reset_index(drop=True)
        
        yes = pd.DataFrame([1]).astype(int)
        data = pd.concat([pd.concat([im1, im2], axis=1, join='inner'), yes],axis=1, join='inner')
        data.columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']
        comb_df = comb_df.append(data)
    #print(comb_df)
    comb_df = comb_df.reset_index(drop=True)
    predictedData, llr = getPredictionResult(comb_df,model)
    
    llr = pd.DataFrame(llr)
    predictedData = pd.DataFrame(predictedData)
    xx = pd.concat([imgList, llr], axis=1, ignore_index=True)
    output_result = pd.concat([xx, predictedData], axis=1, ignore_index=True)
    print(output_result)
    output_result.to_csv('PGMTestOutput.csv', header=['FirstImage','SecondImage', 'LLR','SameOrDifferent'])
    

In [29]:
# path = 'dataset/9feat-dataset/9feat-dataset.csv'
# create_Data(path)

##### Uncomment to run the tests #######
train_path = "comb_df.csv"
#test_path = '/Absolute/FilePath/PGMHiddenTest'
model = PGM(train_path)
#test_two_images(test_path, model)

ValueError: variable names of the model must be identical to column names in data

In [1]:
from pgmpy.models import BayesianModel
import pandas as pd
path='seen-9feat-siamese-pgm-dataset.csv'

#load the previously created data
comb_df = pd.read_csv(path)

#partitioned the data into training data and testing data
train_data = comb_df.iloc[0:10000,:] # if loading from csv [0:3000,1:]
test_data = comb_df.iloc[11000:]

data = pd.DataFrame(train_data)
model = BayesianModel([('f6a', 'f2a'), ('f6a', 'f3a'),('f6a','f4a'),('f3a','f4a'),('f3a','f8a'),('f4a','f1a'),('f4a','f7a'),('f1a','f2a'),('f7a','f1a'),('f7a','f9a'),('f8a','f7a'),('f8a','f5a'),('f5a','f9a'),('f2a','f5a'),('f6b', 'f2b'), ('f6b', 'f3b'),('f6b','f4b'),('f3b','f4b'),('f3b','f8b'),('f4b','f1b'),('f4b','f7b'),('f1b','f2b'),('f7b','f1b'),('f7b','f9b'),('f8b','f7b'),('f8b','f5b'),('f5b','f9b'),('f2b','f5b'),('f7a','verify_ab'),('f7b','verify_ab')])
model.fit(data)
model.get_cpds()

# for cpd in model.get_cpds():
#     print(cpd)

[<TabularCPD representing P(f1a:4 | f4a:5, f7a:4) at 0x10b7e1c18>,
 <TabularCPD representing P(f1b:4 | f4b:5, f7b:4) at 0x10b7e1fd0>,
 <TabularCPD representing P(f2a:5 | f1a:4, f6a:4) at 0x10b7e1e80>,
 <TabularCPD representing P(f2b:5 | f1b:4, f6b:4) at 0x10b7e1cc0>,
 <TabularCPD representing P(f3a:3 | f6a:4) at 0x10b7e1c88>,
 <TabularCPD representing P(f3b:3 | f6b:4) at 0x10b7e1eb8>,
 <TabularCPD representing P(f4a:5 | f3a:3, f6a:4) at 0x10b7e1978>,
 <TabularCPD representing P(f4b:5 | f3b:3, f6b:4) at 0x10b80a390>,
 <TabularCPD representing P(f5a:4 | f2a:5, f8a:5) at 0x10b80a470>,
 <TabularCPD representing P(f5b:4 | f2b:5, f8b:5) at 0x10b80aef0>,
 <TabularCPD representing P(f6a:4) at 0x10b80afd0>,
 <TabularCPD representing P(f6b:4) at 0x10b7e1da0>,
 <TabularCPD representing P(f7a:4 | f4a:5, f8a:5) at 0x10b7e1e48>,
 <TabularCPD representing P(f7b:4 | f4b:5, f8b:5) at 0x10b7e18d0>,
 <TabularCPD representing P(f8a:5 | f3a:3) at 0x10b7e19b0>,
 <TabularCPD representing P(f8b:5 | f3b:3) at 

In [7]:
#predicts the result on test data set
def getPredictionResult(data,model):
    data = data.reset_index(drop=True)
    model_infer = VariableElimination(model)
    out = np.zeros((data.shape[0],1))
    llr = np.zeros((data.shape[0],1))
    arr={'f1b','f2b','f3b','f4b','f5b','f6b','f7b','f8b','f9b'}
    i=0
    for i in range(data.shape[0]):
        #picks 1 image at a time
        image1 = data.iloc[i:i+1,:]
        #set the label i.e node f19 to 1 and predict the values of rest of the nodes from f10-f18
        f1a = int(image1['f1a'][i])
        f2a = int(image1['f2a'][i])
        f3a = int(image1['f3a'][i])
        f4a = int(image1['f4a'][i])
        f5a = int(image1['f5a'][i])
        f6a = int(image1['f6a'][i])
        f7a = int(image1['f7a'][i])
        f8a = int(image1['f8a'][i])
        f9a = int(image1['f9a'][i])
#         result = model_infer.query(['f1b','f2b','f3b','f4b','f5b','f6b','f7b','f8b','f9b'],
#                                    evidence={'f1a':f1a,
#                                              'f2a':f2a,
#                                              'f3a':f3a,
#                                              'f4a':f4a,
#                                              'f5a':f5a,
#                                              'f6a':f6a,
#                                              'f7a':f7a,
#                                              'f8a':f8a,
#                                              'f9a':f9a - 1,
#                                              'verify_ab':0
#                                              })
        result = model_infer.query(['f9a'], evidence={'f1a':f1a,
                                             'f2a':f2a,
                                             'f3a':f3a,
                                             'f4a':f4a,
                                             'f5a':f5a,
                                             'f6a':f6a,
                                             'f7a':f7a,
                                             'f8a':f8a
                                             })
        res_1 = 1;
        #calculate the product of values from predicted Cpd tables by feeding the actual values of image2
#         for x in arr:
#             res_1 = res_1 * (result[x].values[int(image1[x][i])])
        print(result['f9a'])
        break
#         #set the label i.e node f19 to 0 and predict the values of rest of the nodes from f10-f18
#         res_0 = 1;
#         result = model_infer.query(['f10','f11','f12','f13','f14','f15','f16','f17','f18'],evidence={'f1':image1['f1'][i],'f2':image1['f2'][i],'f3':image1['f3'][i],'f4':image1['f4'][i],'f5':image1['f5'][i],'f6':image1['f6'][i],'f7':image1['f7'][i],'f8':image1['f8'][i],'f9':image1['f9'][i],'f19':0})
#         #calculate the product of values from predicted Cpd tables by feeding the actual values of image2
#         for x in arr:
#             res_0 = res_0 * (result[x].values[int(image1[x][i])])
#         #if similar results > dissimilar results prediction is 1 else 0
#         if res_1/res_0 > 1:
#             out[i] = 1;
#         llr[i] = 2 * (np.log(res_1) - np.log(res_0))
    return out,llr

#calculates the accuracy of dataset based on the predicted data and actual data
def calcAccuracy(predictedData,actualData):
    correct_prediction = np.equal(np.round(predictedData), actualData)
    print(np.sum(correct_prediction)/correct_prediction.shape[0])

from pgmpy.inference import VariableElimination
import numpy as np

print("Accuracy for combined DataSet is: ")
predictedData, llr = getPredictionResult(test_data,model)



Accuracy for combined DataSet is: 
+-------+------------+
| f9a   |   phi(f9a) |
+=======+============+
| f9a_0 |     0.0871 |
+-------+------------+
| f9a_1 |     0.9129 |
+-------+------------+


In [19]:

# predictedData = pd.DataFrame(predictedData)
# actualData = test_data['f19'].reshape(predictedData.shape)
# calcAccuracy(predictedData,actualData)

In [1]:
import pandas as pd

img_feats = pd.read_csv('dataset/9feat-dataset/9feat-dataset.csv')
img_pairs = pd.read_csv('dataset/shuffled-dataset/dataset_shuffled_training_siamese.csv')
counter = 0
img_list = img_feats['ImageId'].tolist()

original_feature = ['f1','f2','f3','f4','f5','f6','f7','f8','f9']

comb_df = pd.DataFrame()
comb_df['image_a'] = []
comb_df['image_b'] = []
comb_df['verify_ab'] = []

for f in original_feature:
    comb_df[f + 'a'] = []

for f in original_feature:
    comb_df[f + 'b'] = []

for left_img, right_img  in zip(img_pairs['left'], img_pairs['right']):
    if (left_img[:5] in img_list) and (right_img[:5] in img_list):
        row_list = []
        row_list.append(left_img)
        row_list.append(right_img)
        label = 0
        print(left_img[:4], right_img[:4])
        if(left_img[:4] == right_img[:4]):
            label = 1
        row_list.append(label)
        for f in original_feature:
            row_list.append(int(img_feats.loc[img_feats['ImageId'] == left_img[:5]][f].values[0]))
        for f in original_feature:
            row_list.append(int(img_feats.loc[img_feats['ImageId'] == right_img[:5]][f].values[0]))
        comb_df.loc[counter] = row_list
        counter = counter + 1
# print(features)
# comb_df.to_csv('seen-9feat-siamese-pgm-dataset.csv')

1377 1377
1377 1377
1377 1377
1377 1377
1377 1377
1377 1377
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
0911 0911
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1453 1453
1370 1370
1370 1370
1370 1370
1370 1370
1370 1370
1370 1370
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1271 1271
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
0531 0531
1517 1517
1517 1517
1517 1517
1517 1517
1517 1517
1517 1517
1517 1517
1517 1517
1543 1543
1543 1543
1543 1543
1543 1543
1543 1543
1543 1543
1543 1543
1543 1543
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
0365 0365
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230
1230 1230


0335 0335
0335 0335
0335 0335
0335 0335
0335 0335
0335 0335
0335 0335
0335 0335
0463 0463
0463 0463
0463 0463
0463 0463
0463 0463
0463 0463
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0587 0587
0587 0587
0587 0587
0587 0587
0587 0587
0587 0587
0587 0587
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
1243 1243
1243 1243
1131 1131
1131 1131
1131 1131
1131 1131
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
1477 1477
0595 0595
1153 1153
1153 1153
1153 1153
1153 1153
1153 1153
1153 1153
1153 1153
1153 1153
1153 1153
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0521 0521
0580 0580
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
0489 0489
1496 1496


0496 0496
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0600 0600
0600 0600
0600 0600
0600 0600
0600 0600
0600 0600
0600 0600
0600 0600
0600 0600
0597 0597
0597 0597
0597 0597
0597 0597
0597 0597
0597 0597
0597 0597
0597 0597
0597 0597
1547 1547
1547 1547
1547 1547
1547 1547
0507 0507
0507 0507
0507 0507
0507 0507
0507 0507
0507 0507
0507 0507
0507 0507
0507 0507
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
0337 0337
1375 1375
1375 1375
1375 1375
1375 1375
1375 1375
1375 1375
1375 1375
1375 1375
1198 1198
1198 1198
1198 1198
1198 1198
1198 1198
1198 1198
1198 1198
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
1513 1513
0414 0414
0414 0414
0414 0414
0414 0414
0414 0414
0414 0414
0414 0414
0414 0414
1511 1511
1511 1511
1511 1511
1511 1511
1511 1511
0496 0496
0496 0496
0496 0496
0496 0496
0496 0496
0496 0496
0496 0496
0496 0496
1493 1493


1237 1237
1237 1237
1237 1237
0330 0330
0330 0330
0330 0330
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
1547 1547
1547 1547
1547 1547
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1374 1374
1413 1413
1413 1413
1413 1413
1413 1413
1413 1413
0314 0314
0314 0314
0314 0314
0314 0314
0314 0314
0314 0314
0314 0314
0314 0314
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
0355 0355
0355 0355
0355 0355
0355 0355
0355 0355
0355 0355
0355 0355
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0338 0338
0336 0336
0336 0336
0336 0336
0336 0336
0336 0336
0336 0336
1557 1557
1557 1557
1521 1521
1521 1521
1521 1521
1521 1521
1521 1521
1521 1521
1521 1521
1521 1521
1521 1521
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
0569 0569
1267 1267


1271 1271
1271 1271
1531 1531
1531 1531
1531 1531
1531 1531
1531 1531
1531 1531
1531 1531
1531 1531
0379 0379
0379 0379
0379 0379
0379 0379
0379 0379
0379 0379
0379 0379
0514 0514
0514 0514
0514 0514
0514 0514
0514 0514
0374 0374
0374 0374
0374 0374
0374 0374
0374 0374
0374 0374
1430 1430
1430 1430
1430 1430
1430 1430
1430 1430
1430 1430
1430 1430
1430 1430
1430 1430
0464 0464
1212 1212
1212 1212
1212 1212
1212 1212
1212 1212
1212 1212
0924 0924
0924 0924
0924 0924
0924 0924
0924 0924
0924 0924
0924 0924
0924 0924
1152 1152
1152 1152
1152 1152
1152 1152
1152 1152
1152 1152
1152 1152
0532 0532
1408 1408
1408 1408
1408 1408
1408 1408
1408 1408
1408 1408
1408 1408
1164 1164
1164 1164
1164 1164
1545 1545
1545 1545
1545 1545
1545 1545
1545 1545
1545 1545
1545 1545
1545 1545
0585 0585
0585 0585
0585 0585
0585 0585
0585 0585
0585 0585
0585 0585
0585 0585
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
1497 1497
1497 1497


...

0387 0387
0363 0363
0363 0363
0363 0363
0363 0363
0363 0363
0363 0363
0363 0363
0333 0333
0333 0333
0333 0333
0333 0333
0333 0333
0333 0333
0333 0333
0539 0539
0539 0539
0539 0539
0539 0539
0539 0539
1180 1180
1180 1180
1180 1180
1180 1180
0514 0514
0514 0514
0514 0514
0584 0584
0584 0584
0584 0584
0584 0584
1212 1212
1212 1212
0405 0405
0405 0405
0405 0405
0405 0405
0405 0405
0405 0405
0405 0405
0405 0405
0376 0376
0376 0376
0376 0376
1565 1565
1565 1565
1565 1565
1565 1565
1565 1565
1402 1402
1402 1402
1224 1224
1224 1224
1224 1224
1224 1224
1224 1224
1430 1430
1430 1430
1430 1430
1430 1430
0414 0414
0414 0414
0414 0414
0457 0457
0457 0457
0457 0457
0457 0457
0457 0457
0473 0473
0473 0473
0473 0473
0473 0473
0473 0473
0473 0473
0473 0473
1119 1119
1119 1119
1119 1119
1119 1119
1119 1119
1119 1119
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
0319 0319
1501 1501
1501 1501
1501 1501
1501 1501
1501 1501
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556
0556 0556


1305 1305
1305 1305
0492 0492
0492 0492
0492 0492
0363 0363
0363 0363
0363 0363
0363 0363
0363 0363
0363 0363
1240 1240
1240 1240
1240 1240
1240 1240
1240 1240
1240 1240
1552 1552
1552 1552
1471 1471
1471 1471
0592 0592
0592 0592
0592 0592
0592 0592
1361 1361
1356 1356
1356 1356
1356 1356
1499 1499
1499 1499
1134 1134
1134 1134
1134 1134
1134 1134
1134 1134
1180 1180
1180 1180
1180 1180
1360 1360
1360 1360
1360 1360
0401 0401
0401 0401
0401 0401
0401 0401
0507 0507
0507 0507
0507 0507
0350 0350
0350 0350
0350 0350
0350 0350
0350 0350
0496 0496
0496 0496
0496 0496
0496 0496
1454 1454
1454 1454
1454 1454
1454 1454
1475 1475
1475 1475
1475 1475
1393 1393
1393 1393
1393 1393
1393 1393
1393 1393
1393 1393
1484 1484
1484 1484
1484 1484
1394 1394
1394 1394
1127 1127
1127 1127
1127 1127
1127 1127
1411 1411
1411 1411
1411 1411
1411 1411
1411 1411
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0456 0456
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0535 0535
0315 0315
0315 0315
0315 0315


1123 1123
1123 1123
0507 0507
0507 0507
0472 0472
0472 0472
0472 0472
0472 0472
0472 0472
1344 1344
1344 1344
1344 1344
1344 1344
1181 1181
1181 1181
1181 1181
0348 0348
0333 0333
0333 0333
0333 0333
0333 0333
1236 1236
1236 1236
1236 1236
1236 1236
1476 1476
1430 1430
1137 1137
1137 1137
1137 1137
1137 1137
0585 0585
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
0357 0357
1480 1480
1506 1506
0516 0516
0516 0516
0516 0516
0516 0516
0434 0434
0434 0434
0434 0434
1090 1090
1090 1090
1090 1090
1455 1455
1455 1455
1455 1455
1455 1455
1455 1455
1455 1455
1455 1455
0340 0340
0340 0340
0340 0340
0552 0552
1504 1504
1504 1504
1504 1504
1461 1461
1461 1461
1461 1461
1461 1461
1057 1057
1057 1057
0950 0950
0950 0950
0950 0950
1143 1143
1219 1219
1219 1219
1305 1305
1305 1305
1305 1305
1305 1305
1305 1305
0522 0522
0522 0522
0522 0522
1414 1414
0450 0450
1504 1504
1504 1504
1090 1090
1090 1090
1463 1463
1463 1463
1463 1463
1443 1443
1443 1443
1443 1443
0357 0357
0357 0357
0357 0357


0336 0336
0336 0336
1534 1534
1248 1248
1433 1433
1433 1433
1433 1433
1433 1433
1433 1433
0892 0892
1538 1538
1538 1538
1538 1538
1196 1196
1196 1196
0422 0422
0422 0422
0422 0422
0351 0351
0351 0351
1226 1226
0483 0483
0483 0483
1461 1461
1461 1461
0577 0577
0577 0577
1433 1433
1433 1433
1433 1433
1433 1433
1253 1253
1253 1253
1253 1253
1300 1300
1300 1300
1300 1300
1300 1300
1300 1300
1300 1300
0414 0414
1134 1134
1134 1134
1134 1134
0317 0317
0385 0385
1140 1140
1140 1140
1140 1140
1140 1140
1140 1140
1140 1140
1140 1140
1178 1178
1178 1178
1197 1197
1197 1197
1197 1197
0451 0451
0451 0451
0336 0336
0336 0336
0336 0336
1358 1358
1358 1358
1358 1358
1358 1358
1358 1358
1358 1358
1091 1091
1091 1091
1091 1091
1091 1091
1375 1375
1375 1375
1375 1375
0365 0365
0365 0365
0365 0365
0365 0365
0587 0587
0587 0587
1551 1551
1551 1551
1551 1551
1551 1551
0328 0328
1416 1416
1416 1416
1416 1416
1416 1416
0516 0516
1219 1219
0380 0380
0380 0380
0380 0380
0531 0531
0531 0531
0531 0531
1294 1294


0579 0579
0321 0321
1252 1252
1252 1252
1252 1252
1252 1252
1221 1221
1221 1221
1146 1146
0456 0456
0456 0456
0456 0456
1517 1517
1393 1393
1393 1393
0518 0518
1180 1180
0888 0888
1323 1323
1411 1411
1411 1411
1411 1411
0896 0896
1356 1356
0571 0571
1164 1164
0412 0412
1442 1442
1442 1442
1442 1442
1173 1173
1155 1155
1155 1155
0569 0569
0579 0579
0579 0579
1231 1231
1429 1429
0950 0950
0950 0950
1119 1119
1119 1119
1531 1531
1408 1408
1408 1408
1433 1433
1433 1433
1391 1391
0431 0431
1411 1411
1411 1411
0557 0557
1300 1300
1300 1300
1208 1208
0531 0531
1289 1289
1289 1289
1289 1289
1289 1289
1289 1289
1289 1289
0327 0327
0327 0327
1362 1362
0336 0336
1408 1408
1453 1453
0950 0950
1335 1335
1119 1119
1302 1302
1302 1302
1302 1302
1234 1234
1448 1448
0327 0327
1252 1252
1252 1252
1252 1252
1442 1442
1442 1442
0911 0911
0911 0911
1375 1375
1549 1549
1289 1289
1289 1289
1289 1289
1289 1289
1289 1289
1358 1358
1336 1336
1336 1336
1521 1521
1521 1521
0911 0911
1155 1155
1302 1302
1302 1302
